In [1]:
# Importing necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame

## Problem Statement 1: Ensuring Data Accuracy
**Objective: Ensure that data across different files is accurate and correctly linked.**

Description:

1.	Load Data: Load data from all files into data frames using PySpark.
2.	Initial Validation: Check that data has been ingested correctly into data frames.
3.	Verify Data Accuracy:
    -	Confirm that Customer_ID in transactions.csv, interactions.csv, and customers.csv matches correctly.
    -	Check that Product_ID in transactions.csv is valid according to products.csv.
    -	Ensure that Sales_Rep_ID in transactions.csv matches entries in sales_team.csv.

In [ ]:
def print_green(text: str):
    print(f"\033[92m{text}\033[0m")

def validate_ids(df1: DataFrame, df2: DataFrame, df1_col: str, df2_col: str, id_name: str):
    """
    validate_ids function validates the presence of IDs from one DataFrame in another and displays any missing IDs.

    This function checks whether the IDs from `df1` exist in `df2` by performing a left anti join.
    It then displays the missing IDs in a readable format.

    Parameters:
    df1 (DataFrame): The first DataFrame containing the IDs to be validated.
    df2 (DataFrame): The second DataFrame in which to check for the existence of the IDs from df1.
    df1_col (str): The column name in df1 that contains the IDs.
    df2_col (str): The column name in df2 that contains the corresponding IDs.
    id_name (str): A descriptive name for the ID being validated (e.g., "Customer_ID", "Product_ID").

    Returns:
    None: The function outputs the missing IDs directly using the `show()` method.

    Example:
    ----------
    Validate that all Customer_IDs in transactions_df exist in customers_df
    validate_ids(transactions_df, customers_df, "Customer_ID", "Customer_ID", "Customer_ID")
    
    Output:
    ----------
    Customer_IDs missing in customers.csv from transactions.csv:
    +---------------------+
    | Missing_Customer_ID |
    +---------------------+
    |                   2 |
    |                   6 |
    +---------------------+
    """
    missing_ids = df1.join(df2, df1[df1_col] == df2[df2_col], "left_anti") \
                     .select(df1[df1_col].alias(f"Missing_{id_name}"))
    
    print_green(f"{id_name}s missing in {df2_col.split('_')[0]}.csv from {df1_col.split('_')[0]}.csv:")
    missing_ids.show()

# Step 1: Initialize Spark Session
print_green("Step 1: Initializing Spark Session...")
spark = SparkSession.builder.appName("CRM_Data_Validation").getOrCreate()

# Step 2: Load data from all files into DataFrames using PySpark
print_green("Step 2: Loading data from CSV files into DataFrames...")
customers_df = spark.read.csv("customers.csv", header=True, inferSchema=True)
products_df = spark.read.csv("products.csv", header=True, inferSchema=True)
transactions_df = spark.read.csv("transactions.csv", header=True, inferSchema=True)
interactions_df = spark.read.csv("interactions.csv", header=True, inferSchema=True)
sales_team_df = spark.read.csv("sales_team.csv", header=True, inferSchema=True)

# Step 3: Initial Validation - Display records for each DataFrame
print_green("Step 3: Displaying initial records for validation...")

print_green("Customers DataFrame:")
customers_df.show(5, truncate=False)

print_green("Products DataFrame:")
products_df.show(5, truncate=False)

print_green("Transactions DataFrame:")
transactions_df.show(5, truncate=False)

print_green("Interactions DataFrame:")
interactions_df.show(5, truncate=False)

print_green("Sales Team DataFrame:")
sales_team_df.show(5, truncate=False)

# Step 4: Verifying Data Accuracy

# 4.1 Verify that Customer_ID in transactions.csv and interactions.csv matches with customers.csv
print_green("Step 4.1: Verifying Customer_ID matches across transactions and interactions with customers...")
validate_ids(transactions_df, customers_df, "Customer_ID", "Customer_ID", "Customer_ID")
validate_ids(interactions_df, customers_df, "Customer_ID", "Customer_ID", "Customer_ID")

# 4.2 Check that Product_ID in transactions.csv is valid according to products.csv
print_green("Step 4.2: Verifying Product_ID matches between transactions.csv and products.csv...")
validate_ids(transactions_df, products_df, "Product_ID", "Product_ID", "Product_ID")

# 4.3 Ensure that Sales_Rep_ID in transactions.csv matches entries in sales_team.csv
print_green("Step 4.3: Verifying Sales_Rep_ID matches between transactions.csv and sales_team.csv...")
validate_ids(transactions_df, sales_team_df, "Sales_Rep_ID", "Sales_Rep_ID", "Sales_Rep_ID")

print_green("Data validation completed.")

Step 1: Initializing Spark Session...
Step 2: Loading data from CSV files into DataFrames...
Step 3: Displaying initial records for validation...
Customers DataFrame:
+------------------------------------+-----------------+----------------------+-------------+----------------+
|Customer_ID                         |Name             |Email                 |Phone        |Country         |
+------------------------------------+-----------------+----------------------+-------------+----------------+
|a85e6a90-78d5-490c-a53f-c58b2e57c59b|Shannon Deleon   |NULL                  |5878628895   |Japan           |
|babec972-ffb3-4c56-99c3-e8e3855adf0f|Christina Sanchez|craigprice@example.org|4832368495   |Haiti           |
|d74c33bd-69d9-4718-9e00-d1895a41ddac|Thomas Brown     |vjohnson@example.org  |(276)903-7065|Pakistan        |
|ff05ceba-f459-4714-a252-e03198d9934c|Lindsey Bradford |kathryn50@example.net |NULL         |Marshall Islands|
|f20755f6-8481-4904-afe6-504451ceded5|John Boyer       |